## Ejercicio 5: Automatizar las respuestas a consultas de clientes vía email

Usando como input emails, les damos una primera respuesta automática, adaptándonos al contenido y tono de los mismos. Usaremos un tono corporativo y neutro para ello

In [ ]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
API_VERSION = userdata.get('OPENAI_API_VERSION')
AZURE_ENDPOINT = userdata.get('AZURE_OPENAI_ENDPOINT')

print(type(OPENAI_API_KEY),OPENAI_API_KEY)
print(type(API_VERSION),API_VERSION)
print(type(AZURE_ENDPOINT),AZURE_ENDPOINT)

In [ ]:
from openai import AzureOpenAI
import time

##### Establecemos el valor de la API key de OpenAI. En una aplicación real, no podemos exponer esta API key en el código, y la cogeríamos con una variable de entorno mediante un os.getenv()

### Cogemos un email (aquí ya directamente cargado)

In [ ]:
# Email 1: Consulta sobre un cargo en la cuenta (tono neutro)
email_1 = """\
Asunto: Consulta sobre un cargo en mi cuenta

Estimado equipo de atención al cliente,

He notado que en mi última factura aparece un cargo de 45,00 euros que no reconozco. ¿Podrían proporcionarme información detallada sobre este cargo y el concepto aplicado? Agradezco de antemano su ayuda para aclarar esta situación.

Saludos cordiales,
Carlos Martínez
"""

# Email 2: Consulta sobre horarios de oficina para solicitar una cita
email_2 = """\
Asunto: Solicitud de información sobre horarios y agendamiento de cita

Estimado equipo de atención al cliente,

Me gustaría agendar una cita en una de sus oficinas para revisar algunas cuestiones relacionadas con mis inversiones. ¿Podrían indicarme cuáles son los horarios de atención de sus oficinas y el procedimiento a seguir para programar una cita? Quedo a la espera de su respuesta.

Atentamente,
Lucía Gómez
"""

# Email 3: Reclamación de un cliente muy enfadado
email_3 = """\
Asunto: Reclamación urgente por cargo erróneo

Estoy extremadamente molesto porque se me ha realizado un cargo erróneo en mi cuenta, lo cual ha afectado gravemente mi presupuesto mensual. He intentado comunicarme en varias ocasiones y no he recibido la atención que esperaba. Exijo una solución inmediata y una explicación detallada sobre lo sucedido.

Espero una respuesta YA.

Raúl Sánchez
"""

emails = [email_1, email_2, email_3]
titulos = [
    "Consulta sobre cargo",
    "Solicitud de información sobre horarios y cita",
    "Reclamación urgente"
]

##### Definimos el prompt

In [ ]:
def generar_respuesta(email_text):
    """
    Construye un prompt detallado que le indica al modelo cómo generar una respuesta a un correo electrónico.
    La respuesta debe ser siempre educada, clara, profesional y empática, sin importar el tono del mensaje recibido.
    """
    prompt = (
        "Eres un representante de atención al cliente del banco BBVA España. Tu tarea es generar una respuesta automatizada "
        "a un correo electrónico de un cliente, hablando de usted, y manteniendo siempre un tono educado, profesional, claro y empático. "
        "La respuesta debe incluir:\n"
        "1. Una introducción cordial y agradecimiento por el contacto.\n"
        "2. Una explicación detallada y precisa que responda a las inquietudes planteadas en el correo.\n"
        "3. Instrucciones claras, si es necesario, para resolver el problema o agendar una cita.\n"
        "4. Un cierre amable invitando al cliente a comunicarse nuevamente en caso de tener más dudas.\n\n"
        "No incluyas comentarios personales ni juicios, y asegúrate de que la respuesta sea coherente y profesional.\n\n"
        "A continuación se muestra el correo del cliente:\n"
        "--------------------------\n"
        f"{email_text}\n"
        "--------------------------\n\n"
        "Genera la respuesta al correo:"
    )
    return prompt

In [ ]:
def generar_respuesta_pirata(email_text):
    """
    Construye un prompt detallado que le indica al modelo cómo generar una respuesta a un correo electrónico.
    La respuesta debe ser siempre educada, clara, profesional y empática, sin importar el tono del mensaje recibido.
    """
    prompt = (
        "Eres un representante de atención al cliente del banco BBVA España. Tu tarea es generar una respuesta automatizada "
        "a un correo electrónico de un cliente, hablando como un pirata en tono de broma."
        "La respuesta debe incluir:\n"
        "1. Una introducción como rAImundo con variaciones del texto 'el mejor y más guapo pirata de los siete mares de las finanzas' o algo igualmente ridículo\n"
        "2. Una explicación hablando como un pirata a las inquietudes planteadas en el correo, adoptando un tono financiero de la época pirata\n"
        "3. Instrucciones claras, si es necesario, para resolver la situación como un pirata (duelos al amanecer, tirarle por la borda, etc.).\n"
        "4. Un cierre amenazante invitando al cliente a una batalla en alta mar con cañones.\n\n"
        "Asegúrate de que la respuesta sea muy divertida.\n\n"
        "A continuación se muestra el correo del cliente:\n"
        "--------------------------\n"
        f"{email_text}\n"
        "--------------------------\n\n"
        "Genera la respuesta al correo:"
    )
    return prompt

##### Generamos las respuestas

In [ ]:
from IPython.display import Markdown

client = AzureOpenAI(
            api_key = OPENAI_API_KEY,
            api_version=API_VERSION,
            azure_endpoint= AZURE_ENDPOINT# Este endpoint corresponde a la región france-central
)

for idx, email_text in enumerate(emails):
    print(f"\n--- Respuesta para el Email {idx+1} ({titulos[idx]}) ---\n")
    prompt_final = generar_respuesta(email_text)
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt_final}],
        temperature=0.3,
        max_tokens=500,
    )
    respuesta = response.choices[0].message.content.strip()
    display(Markdown(respuesta))
    print("\n\n")
    time.sleep(1)  # Pausa para evitar problemas con rate limits

In [ ]:
for idx, email_text in enumerate(emails):
    print(f"\n--- Respuesta para el Email {idx+1} ({titulos[idx]}) ---\n")
    prompt_final = generar_respuesta_pirata(email_text)
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt_final}],
        temperature=0.7,
        max_tokens=500,
    )
    respuesta = response.choices[0].message.content.strip()
    display(Markdown(respuesta))
    print("\n\n")
    time.sleep(1)  # Pausa para evitar problemas con rate limits

##### - ¿Cómo varía el tono y la estructura de la respuesta generada en función del correo recibido?
##### - ¿Consideras que el prompt es lo suficientemente detallado para garantizar una respuesta profesional y empática? ¿Qué ajustes sugerirías?
##### - ¿De qué manera influye la temperatura en la calidad de la respuesta generada?